# Инструменты машинного обучения в Earth Engine 

В данном блокноте демонстрируются основные приемы работы с GEE на примере решения задачи средствами Earth Engine.

## Инициализация

Выполняетя по необходимости, в случае если не была выполнена ранее или в случае возникновения тайм-аута.

In [1]:
import numpy as np
import sklearn as sk

import ee
from IPython.display import Image

try:
  ee.Initialize()
  print('Earth Engine успешно инициализирован!')
except ee.EEException:
  print('The Earth Engine package failed to initialize!')


Earth Engine успешно инициализирован!


In [3]:
# !earthengine authenticate

In [4]:
# !earthengine authenticate --authorization-code="4/AwGGCrx2piS9PytjNmThMQgjbZNR6lIkGJMddPW0e_D4DL2phjGu5so"

In [2]:
!earthengine asset info users/kolesovdm/DVFU

{
  "id": "users/kolesovdm/DVFU",
  "type": "Folder"
}


In [3]:
!earthengine ls -l users/kolesovdm/DVFU

[Table]  users/kolesovdm/DVFU/simpleDataDVFU
[Table]  users/kolesovdm/DVFU/simpleTestDataDVFU


## Обработка данных средствами Python API для Google Earth Engine. 

### Базовый пример

После инициализации становятся доступны функции Python API. Прочитаем, например содержимое таблицы `users/kolesovdm/DVFU/simpleDataDVFU`:

In [4]:
sample = ee.FeatureCollection('users/kolesovdm/DVFU/simpleDataDVFU')

Обратим внимание, что объект `sample` не содержит сами данные, на самом деле он связан с данными, лежащими на серверах Google Earth Engine. Все манипуляции с объектами Earth Engine производятся удаленно, поскольку код, использующий Python API для Earth Engine при выполнении передается на сервера Google, где компилируется во внутренее представление, а затем выполняется. 

In [5]:
print(sample)

ee.FeatureCollection({
  "type": "Invocation", 
  "arguments": {
    "tableId": "users/kolesovdm/DVFU/simpleDataDVFU"
  }, 
  "functionName": "Collection.loadTable"
})


Тем не менее есть возможность получить эти данные локально при помощи функции `.getInfo()`, но поступать так следует с осторожностью, т.к. передача данных по сети и/или выполнение программного кода локально очень замедляет процесс обработки.

In [7]:
# sample.getInfo()

### Обзор основных приемов работы с GEE

#### Доступные для анализа данные

Спутниковая съемка:
 * [Landsat](https://developers.google.com/earth-engine/datasets/catalog/landsat/)
 * [Sentinel](https://developers.google.com/earth-engine/datasets/catalog/sentinel/)
 * [MODIS](https://developers.google.com/earth-engine/datasets/catalog/modis)
 
 
[Аэрофотосъемка и съемка высокого разрешения](https://developers.google.com/earth-engine/datasets/tags/highres)

Климатические данные:
 * [Температуры](https://developers.google.com/earth-engine/datasets/tags/temperature)
 * [Общеклиматические данные](https://developers.google.com/earth-engine/datasets/tags/climate)
 * [Метеорологические данные](https://developers.google.com/earth-engine/datasets/tags/weather)
 * [Параметры атмосферы](https://developers.google.com/earth-engine/datasets/tags/atmosphere)
 

Геофизические данные:
 * [Высоты (цифровые модели местности)](https://developers.google.com/earth-engine/datasets/tags/elevation)
 * [Классы земного покрова (такие как лес, луг, вода, ...)](https://developers.google.com/earth-engine/datasets/tags/landcover)
 
 
Пользователь может добавлять/импортировать собственные данные (ASSETS).

##### Пример доступа к данным

У каждой коллекции данных есть свой идентификатор, по которому можно ее получить. Например, Sentinel-2 доступен по ссылке с идентификатором `COPERNICUS/S2`:

In [8]:
sentinel = ee.ImageCollection("COPERNICUS/S2")
print(sentinel)

ee.ImageCollection({
  "type": "Invocation", 
  "arguments": {
    "id": "COPERNICUS/S2"
  }, 
  "functionName": "ImageCollection.load"
})


Аналогично, у каждого изображения также есть собственный идентификатор, по которому можно сослаться на это изображение:

In [9]:
# Получим ссылку на изображение
my_img = ee.Image('COPERNICUS/S2/20181002T020651_20181002T021217_T52TGN');

# Отрисуем его
Image(url=my_img.getThumbUrl({'min': 0, 'max': 1500, 'bands': 'B7,B5,B3'}))

In [10]:
region = ee.Geometry.Rectangle([131.74, 42.95, 131.94, 43.07])
my_img = ee.Image('COPERNICUS/S2/20181002T020651_20181002T021217_T52TGN').clip(region)
Image(url=my_img.getThumbUrl({'min': 0, 'max': 2000, 'bands': 'B2,B5,B3'}))

#### Обзор API

##### Основные структуры данных

Два основных типа данных в GEE:
 * Image: растровый объект, состоит из каналов изображения и словаря свойств.
 * Feature: векоторный объект, состоит из геометрии (тип `Geometry`) и словаря свойств.
 
Производные от данных типов, например:
 * ImageCollection: стек изображений (растров).
 * FeatureCollection: список отдельных векторных объектов.

Аналогичные стандартным, знакомым по другим языкам (строки, числа и т.п.) типам:
 * Dictionary: словарь.
 * List: список.
 * Array: массив.
 * Geometry: геометрия.
 * Date: дата. 
 * Number: числовой тип.
 * String: строки.
У многих этих типов есть подтипы  (например, у числового - byte, float64 и т.п.).

Пример: отобразим свойства изображения.

In [11]:
prop = my_img.getInfo()
print(prop.keys())
# print(prop['properties'])
# print(prop['properties']['MEAN_INCIDENCE_ZENITH_ANGLE_B10']) # Зенитный угол Солнца в момент съемки

[u'bands', u'version', u'type', u'id', u'properties']


##### Основные алгоритмы

**Все перечисленные выше объекты являются объектами, хранящимися и обрабатываемыми на серверах Google.** Код, который вы пишите, отправляется на сервер, там компилируется и запускается. Объекты не обрабатываются локально.

GEE поддерживает множество встроенных алгоритмов обработки (гео)данных. Но нас будут интересовать, в первую очередь, алгоритмы машинного обучения:
 * Алогоритмы классификации:
  - Деревья решений.
  - Случайный лес.
  - Машины опорных векторов.
  - Байесовский классификатор.
  - ....
 * Алгоритмы кластеризации (в основном модификации К-средних).
 * Статистические методы:
  - Регрессии (классическая линейная, взвешенная, робастная, ...).
  - Статистические метрики качества классификаторов и сходства выборок.
 * ...
  

### Пример

Обучение с учителем доступно при использовании пакета `Classifier`. Общая схема работы:

1. Создание обучающих данных. Необходимо собрать векторные объекты, на основе которых будет производиться обучение. В атрибутах объектов должно быть числовое поле - номер класса объекта.
2. Создать объект классификатора и установить его параметры.
3. Обучить классификатор на обучающих данных.
4. Подать обученному классификатору входные данные в виде изображений или колеекции объектов.
5. Оценить ошибку классификации на независимой выборке.



### Исходные данные

Возьмем для работы в качестве источника изображений спутник Sentinel-2. Построим классификатор, который будет  определять по изображению следующие типы Земного покрова:
 * Водные объекты (море).
 * Растительность.
 * Городская застройка.
 
*(Это иллюстративный пример, типы объектов очень грубые, в реальности все сложнее)*

##### Каналы Sentinel-2

Название|Масштабный коэффициент|Разрешение|Длина волны|Краткое описание
---|-----|----------|-----------|-----------
B1|0.0001|60 m |443.9nm (S2A) / 442.3nm (S2B)|Аэрозоли
B2|0.0001|10 m |496.6nm (S2A) / 492.1nm (S2B)|Синий
B3|0.0001|10 m |560nm (S2A) / 559nm (S2B)|Зеленый
B4|0.0001|10 m |664.5nm (S2A) / 665nm (S2B)|Красный
B5|0.0001|20 m |703.9nm (S2A) / 703.8nm (S2B)|Граница красного-1
B6|0.0001|20 m |740.2nm (S2A) / 739.1nm (S2B)|Граница красного-2
B7|0.0001|20 m |782.5nm (S2A) / 779.7nm (S2B)|Граница красного-3
B8|0.0001|10 m |835.1nm (S2A) / 833nm (S2B)|Ближний инфракрасный
B8a|0.0001|20 m |864.8nm (S2A) / 864nm (S2B)|Граница красного-4 
B9|0.0001|60 m |945nm (S2A) / 943.2nm (S2B)|Водяной пар
B10|0.0001|60 m |1373.5nm (S2A) / 1376.9nm (S2B)|Перистые облака
B11|0.0001|20 m |1613.7nm (S2A) / 1610.4nm (S2B)|Коротковолновый инфракрасный-1 
B12|0.0001|20 m |2202.4nm (S2A) / 2185.7nm (S2B)|Коротковолновый инфракрасный-2 

Будем использовать для анализа каналы видимой части спектра, ближний инфракрасный и коротковолновый инфракрасный: 'B2', 'B3', 'B4', 'B5', 'B8', 'B11':

In [12]:
bands = ['B2', 'B3', 'B4', 'B8', 'B11'];

Выберем изображение для анализа:

In [13]:
image = ee.Image('COPERNICUS/S2/20181002T020651_20181002T021217_T52TGN')

#### Обучающие полигоны
Отрисуем обучающие полигоны и импортируем их в GEE. (Об этом процессе рассказывается в параллельном курсе.) Коротко:
 * рисуем полигоны, каждый полигон - объект одного класса;
 * каждому полигону прописываем в атрибутах его класс (номер класса);
 * импортируем в GEE при помощи графического интерфейса https://code.earthengine.google.com: Вкладка Assests -> Кнопка New -> Пункт меню Table Upload -> В появившемся окне выбираем файлы для загрузки и указываем название результата импорта (название таблицы с геоданными).

В итоге с локального компьютера будут загружены файлы (например, shp) и размещены под указанным названием.

В Asset под названием `users/kolesovdm/DVFU` был предварительно импортирован слой с примерами `simpleDataDVFU`: https://code.earthengine.google.com/541b416bec25ec1c3cf42669fdec0fc4

Обучающие полигоны:

In [14]:
trainData = ee.FeatureCollection('users/kolesovdm/DVFU/simpleDataDVFU');

In [15]:
# Загружаем обучающие данные. Нумерация классов должна начинаться с нуля.
# Номер класса хранится в поле 'ClassNumber'
remapped = trainData.remap([1, 2, 3], [0, 1, 2], 'ClassNumber');
# print(remapped);

Полигоны разобъем на отдельные пиксели, накрывающие изображение, и прочитаем содержимое этих пикселей.

In [16]:
training = image.sampleRegions(
  collection=remapped,
  properties=['ClassNumber'],
  scale=10
);

# print(training.size().getInfo())

#### Классификатор: CART 
CART == Classification and Regression Trees

Реализация CART в GEE использует следующие параметры:

* crossvalidationFactor (по умолчанию 10): число разбиений при перекрестной проверке;
* maxDepth (по умолчанию 10):  максимальная глубина дерева;
* minLeafPopulation (по умолчанию 1): не допускается создание листьев с числом объектов меньших, чем minLeafPopulation;
* minSplitPoplulation (по умолчанию 1): не разбивать узлы, если число объектов узла меньше или равно minSplitPoplulation;
* minSplitCost (по умолчанию 1e-10): не разбивать узел, если ошибка (цена поддерева, зависящая от ошибки) меньше, чем minSplitCost;
* prune (по умолчанию false): пропускать ли процедуру обрезки дерева;
* pruneErrorTolerance (по умолчанию 0.5): The standard error threshold to use in determining the simplest tree whose accuracy is comparable to the minimum cost-complexity tree;
* quantizationResolution (по умолчанию 100): The quantization resolution for numerical features;
* quantizationMargin (по умолчанию 0.1): The margin reserved by quantizer to avoid overload, as a fraction of the range observed in the training data;
* randomSeed (по умолчанию 0): начальное значение для генератора псевдослучайных чисел.

##### Обучение

Обучим классификатор CART:

In [17]:
classifier = ee.Classifier.cart(
  prune=True,
  maxDepth=3
).train(training, 'ClassNumber', bands);

Построим результат классификации:

In [18]:
classified = image.select(bands).classify(classifier);

И отобразим его:

In [19]:
Image(url=classified.getThumbUrl())

Выглядит не очень понятно.

Расскрасим изображение цветовой палитрой (по коду класса):
 * Зеленый (0)
 * Синий (1)
 * Красный (2)

In [20]:
visualParams = {'min': 0, 'max': 2, 'palette': '00FF00,0000FF,FF0000'}
Image(url=classified.getThumbUrl(visualParams))

Посмотрим на результат в более крупном масштабе:

In [21]:
region = ee.Geometry.Rectangle([131.74, 42.95, 131.94, 43.07])
clipped = classified.clip(region) 
Image(url=clipped.getThumbUrl(visualParams))

##### Оценка точности

Посчитаем матрицу ошибок сначала на обучающем множестве, затем на тестовом и сравним результаты.

Обучающее множество:

In [22]:
trainAccuracy = classifier.confusionMatrix()
matrix = trainAccuracy.getInfo()
print('Матрица ошибок на обучающем множестве:')
print(np.array(matrix))
acc = trainAccuracy.accuracy().getInfo()
print('Точность на обучающем множестве:')
print(acc)

Матрица ошибок на обучающем множестве:
[[ 23438      0      2]
 [     0 100711     39]
 [    13     22   1160]]
Точность на обучающем множестве:
0.99939386689


Тестовое множество. Повторим процедуру -- реклассификация меток класса, выборка данных и матрица ошибок.

In [23]:
testData = ee.FeatureCollection('users/kolesovdm/DVFU/simpleTestDataDVFU');
remappedTest = testData.remap([1, 2, 3], [0, 1, 2], 'ClassNumber')

# Сделаем выборку
scale = 60  # При хорошем разрешении упираемся в ошибку time-out'а, поэтому огрубим результаты
testAnswers = image.select(bands).sampleRegions(
  collection=remappedTest,
  properties=['ClassNumber'],
  scale=scale
  ).classify(classifier)


# Get a confusion matrix representing expected accuracy.
testAccuracy = testAnswers.errorMatrix('ClassNumber', 'classification');

print('Матрица ошибок на тестовом множестве: ')
test_matrix = np.array(testAccuracy.getInfo())
print(test_matrix)
print('Точность на тестовом множестве: ')
test_acc = testAccuracy.accuracy().getInfo()
print(test_acc)

Матрица ошибок на тестовом множестве: 
[[ 1028     0     0]
 [    0 11098     1]
 [    4     0   129]]
Точность на тестовом множестве: 
0.999592169657


## Задание

1. Выбрать снимок на интересующую территорию.
2. Создать обучающее и тестовые множества.
3. Построить и обучить классификатор.
4. Оценить качество работы классификатора.